In [1]:
!pip install statsmodels

In [2]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import gc
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

In [3]:
all_data_df = pd.read_csv("./Data/Data.csv")
ss = pd.read_csv("SampleSubmission.csv")

In [4]:
len(ss)

6014

In [5]:
all_data_df.columns

Index(['date_time', 'v_red', 'current', 'power_factor', 'kwh', 'Source',
       'v_blue', 'v_yellow', 'consumer_device_9', 'consumer_device_x'],
      dtype='object')

In [6]:
all_data_df.head()

,date_time,v_red,current,power_factor,kwh,Source,v_blue,v_yellow,consumer_device_9,consumer_device_x
0,2024-07-22 18:20:00,137.65,0.08,0.72,0.000661,consumer_device_10_data_user_1,NaN,NaN,0,10
1,2024-07-22 18:25:00,122.82,0.08,0.73,0.000598,consumer_device_10_data_user_1,NaN,NaN,0,10
2,2024-07-22 18:30:00,119.70,0.08,0.74,0.000591,consumer_device_10_data_user_1,NaN,NaN,0,10
3,2024-07-22 18:35:00,124.53,0.08,0.75,0.000623,consumer_device_10_data_user_1,NaN,NaN,0,10
4,2024-07-22 18:40:00,134.84,0.08,0.74,0.000665,consumer_device_10_data_user_1,NaN,NaN,0,10


In [7]:
# Split 'Source' into 'consumer_device_X' and 'data_user_Y'
all_data_df[['consumer_device', 'data_user']] = all_data_df['Source'].str.extract(r'(consumer_device_\d+)_data_user_(\d+)')

# Display the updated DataFrame (optional)
all_data_df.head()

,date_time,v_red,current,power_factor,kwh,Source,v_blue,v_yellow,consumer_device_9,consumer_device_x,consumer_device,data_user
0,2024-07-22 18:20:00,137.65,0.08,0.72,0.000661,consumer_device_10_data_user_1,NaN,NaN,0,10,consumer_device_10,1
1,2024-07-22 18:25:00,122.82,0.08,0.73,0.000598,consumer_device_10_data_user_1,NaN,NaN,0,10,consumer_device_10,1
2,2024-07-22 18:30:00,119.70,0.08,0.74,0.000591,consumer_device_10_data_user_1,NaN,NaN,0,10,consumer_device_10,1
3,2024-07-22 18:35:00,124.53,0.08,0.75,0.000623,consumer_device_10_data_user_1,NaN,NaN,0,10,consumer_device_10,1
4,2024-07-22 18:40:00,134.84,0.08,0.74,0.000665,consumer_device_10_data_user_1,NaN,NaN,0,10,consumer_device_10,1


In [9]:
# These are the devices that are not in the test SampleSubmission

devices_to_drop = ["consumer_device_3","consumer_device_5","consumer_device_11", "consumer_device_14",
                   "consumer_device_15", "consumer_device_17", "consumer_device_24",
                   "consumer_device_25","consumer_device_27","consumer_device_33","consumer_device_4","consumer_device_9"]

In [10]:
# Filter the DataFrame to include only rows where 'consumer_device' is in the 'devices_to_drop' list.
filtered_df = all_data_df[all_data_df['consumer_device'].isin(devices_to_drop)]

# Now 'filtered_df' contains only the rows you specified.  You can further process or save this DataFrame.
filtered_df.head()

,date_time,v_red,current,power_factor,kwh,Source,v_blue,v_yellow,consumer_device_9,consumer_device_x,consumer_device,data_user
327816,2023-10-01 14:35:00,169.26,3.50,0.73,0.036038,consumer_device_11_data_user_1,NaN,NaN,0,11,consumer_device_11,1
327817,2023-10-01 14:40:00,169.20,3.15,0.76,0.033755,consumer_device_11_data_user_1,NaN,NaN,0,11,consumer_device_11,1
327818,2023-10-01 14:45:00,168.38,2.58,0.73,0.026427,consumer_device_11_data_user_1,NaN,NaN,0,11,consumer_device_11,1
327819,2023-10-01 14:50:00,168.87,2.52,0.76,0.026952,consumer_device_11_data_user_1,NaN,NaN,0,11,consumer_device_11,1
327820,2023-10-01 14:55:00,168.30,2.47,0.75,0.025981,consumer_device_11_data_user_1,NaN,NaN,0,11,consumer_device_11,1


In [149]:
# for all_data["Source"] aggregate by sum on day

import pandas as pd
# Assuming 'all_data_df' is already defined as in your previous code.
# Convert 'Datetime' column to datetime objects if it's not already
all_data_df['date_time'] = pd.to_datetime(all_data_df['date_time'])

# Extract the date part
all_data_df['Date'] = all_data_df['date_time'].dt.date

aggregated_data = all_data_df.groupby(['Source', 'Date']).agg({
    'kwh': 'sum',  # Sum of kwh
    'power_factor': 'mean'  # Average power factor
    
}).reset_index()
# Display the aggregated data
aggregated_data


,Source,Date,kwh,power_factor
0,consumer_device_10_data_user_1,2024-07-22,0.024330,0.384118
1,consumer_device_10_data_user_1,2024-07-23,0.103560,0.342986
2,consumer_device_10_data_user_1,2024-07-24,0.137543,0.399896
3,consumer_device_10_data_user_1,2024-07-25,0.121011,0.356250
4,consumer_device_10_data_user_1,2024-07-26,0.000000,0.000000
...,...,...,...,...
136404,consumer_device_9_data_user_9,2024-09-10,0.000000,0.000000
136405,consumer_device_9_data_user_9,2024-09-11,0.000000,0.000000
136406,consumer_device_9_data_user_9,2024-09-12,0.000000,0.000000
136407,consumer_device_9_data_user_9,2024-09-13,0.000000,0.000000


In [150]:
# Find the minimum and maximum date_time values
min_date = aggregated_data['Date'].min()
max_date = aggregated_data['Date'].max()

print(f"Minimum date_time: {min_date}")
print(f"Maximum date_time: {max_date}")

Minimum date_time: 2023-06-03
Maximum date_time: 2024-09-23


In [151]:
# Fill missing date values with 0 kwh

# Create a date range
date_rng = pd.date_range(start=min_date, end=max_date, freq='D')

# Create an empty DataFrame to store the complete data
complete_data = pd.DataFrame()

# Iterate through each unique 'Source'
for source in aggregated_data['Source'].unique():
    # Extract data for the current 'Source'
    source_data = aggregated_data[aggregated_data['Source'] == source].copy()

    # Convert the source data Date to match the type of date_rng
    source_data['Date'] = pd.to_datetime(source_data['Date'])

    # Create a complete date range for the current 'Source'
    source_date_rng = pd.DataFrame({'Date': date_rng})
    source_date_rng['Source'] = source

    # Merge with the existing data, filling missing 'kwh' values with 0
    source_data = pd.merge(source_date_rng, source_data, on=['Date', 'Source'], how='left')
    source_data = source_data.dropna()

    # Append to the complete data
    complete_data = pd.concat([complete_data, source_data], ignore_index=True)

In [152]:
complete_data.head()

,Date,Source,kwh,power_factor
0,2024-07-22,consumer_device_10_data_user_1,0.024330,0.384118
1,2024-07-23,consumer_device_10_data_user_1,0.103560,0.342986
2,2024-07-24,consumer_device_10_data_user_1,0.137543,0.399896
3,2024-07-25,consumer_device_10_data_user_1,0.121011,0.356250
4,2024-07-26,consumer_device_10_data_user_1,0.000000,0.000000


In [153]:
### CHANGE NAME OF FOLDER AND FILES TO CORRESPOND ####
climate_df = pd.read_excel("./ClimateData/Climate Data/Kalam Climate Data.xlsx")# Convert to datetime
complete_data["Date"] = pd.to_datetime(complete_data["Date"])
climate_df["Date Time"] = pd.to_datetime(climate_df["Date Time"])

# Aggregate climate data to daily level
climate_daily = climate_df.groupby(climate_df["Date Time"].dt.date).agg({
    "Temperature (°C)": "mean",
    "Dewpoint Temperature (°C)": "mean",
    "U Wind Component (m/s)": "mean",
    "V Wind Component (m/s)": "mean",
    "Total Precipitation (mm)": "sum",
    "Snowfall (mm)": "sum",
    "Snow Cover (%)": "mean",
}).reset_index()

# Convert 'Date' column in climate_daily to datetime format
climate_daily.rename(columns={"Date Time": "Date"}, inplace=True)
climate_daily["Date"] = pd.to_datetime(climate_daily["Date"])  # Ensure datetime64[ns]

# Merge with complete_data
complete_data = complete_data.merge(climate_daily, on="Date", how="left")

In [154]:
import pandas as pd
import numpy as np

# Convert 'Date' to datetime format if not already
complete_data["Date"] = pd.to_datetime(complete_data["Date"])

# Extract date features (keeping only useful ones)
complete_data["year"] = complete_data["Date"].dt.year
complete_data["day"] = complete_data["Date"].dt.day
complete_data["is_weekend"] = (complete_data["Date"].dt.dayofweek >= 5).astype(int)  # 1 if Sat/Sun, else 0

# Sinusoidal encoding of cyclical features
complete_data["day_of_week_sin"] = np.sin(2 * np.pi * complete_data["Date"].dt.dayofweek / 7)
complete_data["day_of_week_cos"] = np.cos(2 * np.pi * complete_data["Date"].dt.dayofweek / 7)

complete_data["month_sin"] = np.sin(2 * np.pi * complete_data["Date"].dt.month / 12)
complete_data["month_cos"] = np.cos(2 * np.pi * complete_data["Date"].dt.month / 12)

complete_data["week_of_year_sin"] = np.sin(2 * np.pi * complete_data["Date"].dt.isocalendar().week / 52)
complete_data["week_of_year_cos"] = np.cos(2 * np.pi * complete_data["Date"].dt.isocalendar().week / 52)

complete_data["quarter_sin"] = np.sin(2 * np.pi * complete_data["Date"].dt.quarter / 4)
complete_data["quarter_cos"] = np.cos(2 * np.pi * complete_data["Date"].dt.quarter / 4)



In [155]:
# Extract consumer device and data user using regex
complete_data["consumer_device"] = complete_data["Source"].str.extract(r'consumer_device_(\d+)_data_user_\d+').astype(int)
complete_data["data_user"] = complete_data["Source"].str.extract(r'consumer_device_\d+_data_user_(\d+)').astype(int)

In [156]:
complete_data

,Date,Source,kwh,power_factor,Temperature (°C),Dewpoint Temperature (°C),U Wind Component (m/s),V Wind Component (m/s),Total Precipitation (mm),Snowfall (mm),...,day_of_week_sin,day_of_week_cos,month_sin,month_cos,week_of_year_sin,week_of_year_cos,quarter_sin,quarter_cos,consumer_device,data_user
0,2024-07-22,consumer_device_10_data_user_1,0.024330,0.384118,14.719596,8.280669,0.022655,-0.104998,0.073049,0.000000,...,0.000000,1.000000,-0.5,-8.660254e-01,-0.464723,-0.885456,-1.0,-1.836970e-16,10,1
1,2024-07-23,consumer_device_10_data_user_1,0.103560,0.342986,13.217268,9.862700,0.116137,-0.077997,0.121921,0.000000,...,0.781831,0.623490,-0.5,-8.660254e-01,-0.464723,-0.885456,-1.0,-1.836970e-16,10,1
2,2024-07-24,consumer_device_10_data_user_1,0.137543,0.399896,12.462190,9.865658,0.103451,-0.075028,0.119984,0.000000,...,0.974928,-0.222521,-0.5,-8.660254e-01,-0.464723,-0.885456,-1.0,-1.836970e-16,10,1
3,2024-07-25,consumer_device_10_data_user_1,0.121011,0.356250,13.867551,8.973798,0.066345,-0.043182,0.034283,0.000000,...,0.433884,-0.900969,-0.5,-8.660254e-01,-0.464723,-0.885456,-1.0,-1.836970e-16,10,1
4,2024-07-26,consumer_device_10_data_user_1,0.000000,0.000000,15.572609,9.434734,0.093025,0.046414,0.006961,0.000000,...,-0.433884,-0.900969,-0.5,-8.660254e-01,-0.464723,-0.885456,-1.0,-1.836970e-16,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136404,2024-09-10,consumer_device_9_data_user_9,0.000000,0.000000,8.972100,4.735042,0.100475,0.099872,0.012961,0.000000,...,0.781831,0.623490,-1.0,-1.836970e-16,-0.970942,-0.239316,-1.0,-1.836970e-16,9,9
136405,2024-09-11,consumer_device_9_data_user_9,0.000000,0.000000,9.216674,2.945390,0.031060,-0.080538,0.014010,0.000000,...,0.974928,-0.222521,-1.0,-1.836970e-16,-0.970942,-0.239316,-1.0,-1.836970e-16,9,9
136406,2024-09-12,consumer_device_9_data_user_9,0.000000,0.000000,10.711587,1.898337,-0.069007,-0.089148,0.002685,0.000000,...,0.433884,-0.900969,-1.0,-1.836970e-16,-0.970942,-0.239316,-1.0,-1.836970e-16,9,9
136407,2024-09-13,consumer_device_9_data_user_9,0.000000,0.000000,10.078417,3.285115,0.170258,0.271345,0.009149,0.000000,...,-0.433884,-0.900969,-1.0,-1.836970e-16,-0.970942,-0.239316,-1.0,-1.836970e-16,9,9


In [157]:
#complete_data.drop(columns=["v_blue","v_red","v_yellow","current","kwh"], inplace=True)

X_power = complete_data.drop(columns=['Date', 'Source', 'power_factor','kwh'])
y_power = complete_data.power_factor
X = complete_data.drop(columns=['Date', 'Source', 'kwh',])
y = complete_data.kwh

In [158]:
X_power.fillna(-1,inplace=True)
X.fillna(-1,inplace=True)
X_power

,Temperature (°C),Dewpoint Temperature (°C),U Wind Component (m/s),V Wind Component (m/s),Total Precipitation (mm),Snowfall (mm),Snow Cover (%),year,day,is_weekend,day_of_week_sin,day_of_week_cos,month_sin,month_cos,week_of_year_sin,week_of_year_cos,quarter_sin,quarter_cos,consumer_device,data_user
0,14.719596,8.280669,0.022655,-0.104998,0.073049,0.000000,0.000000,2024,22,0,0.000000,1.000000,-0.5,-8.660254e-01,-0.464723,-0.885456,-1.0,-1.836970e-16,10,1
1,13.217268,9.862700,0.116137,-0.077997,0.121921,0.000000,0.000000,2024,23,0,0.781831,0.623490,-0.5,-8.660254e-01,-0.464723,-0.885456,-1.0,-1.836970e-16,10,1
2,12.462190,9.865658,0.103451,-0.075028,0.119984,0.000000,0.000000,2024,24,0,0.974928,-0.222521,-0.5,-8.660254e-01,-0.464723,-0.885456,-1.0,-1.836970e-16,10,1
3,13.867551,8.973798,0.066345,-0.043182,0.034283,0.000000,0.000000,2024,25,0,0.433884,-0.900969,-0.5,-8.660254e-01,-0.464723,-0.885456,-1.0,-1.836970e-16,10,1
4,15.572609,9.434734,0.093025,0.046414,0.006961,0.000000,0.000000,2024,26,0,-0.433884,-0.900969,-0.5,-8.660254e-01,-0.464723,-0.885456,-1.0,-1.836970e-16,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136404,8.972100,4.735042,0.100475,0.099872,0.012961,0.000000,0.000000,2024,10,0,0.781831,0.623490,-1.0,-1.836970e-16,-0.970942,-0.239316,-1.0,-1.836970e-16,9,9
136405,9.216674,2.945390,0.031060,-0.080538,0.014010,0.000000,0.000000,2024,11,0,0.974928,-0.222521,-1.0,-1.836970e-16,-0.970942,-0.239316,-1.0,-1.836970e-16,9,9
136406,10.711587,1.898337,-0.069007,-0.089148,0.002685,0.000000,0.000000,2024,12,0,0.433884,-0.900969,-1.0,-1.836970e-16,-0.970942,-0.239316,-1.0,-1.836970e-16,9,9
136407,10.078417,3.285115,0.170258,0.271345,0.009149,0.000000,0.000000,2024,13,0,-0.433884,-0.900969,-1.0,-1.836970e-16,-0.970942,-0.239316,-1.0,-1.836970e-16,9,9


In [159]:
from sklearn.model_selection import *

X_train_power, X_test_power, y_train_power, y_test_power = train_test_split(X_power, y_power, random_state=42, 
                                                    shuffle=True, stratify=X_power['consumer_device'])

In [160]:
from sklearn.model_selection import *

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, 
                                                    shuffle=True, stratify=X['consumer_device'])

## PREDICTION power_factor

In [161]:
from catboost import *
model_power = CatBoostRegressor(iterations=10000, loss_function='RMSE', use_best_model=True,
                          early_stopping_rounds=100, verbose=1000, random_state=42,
                          )
model_power.fit(X_train_power, y_train_power, eval_set=(X_test_power, y_test_power))

Learning rate set to 0.025849
0:	learn: 0.1587110	test: 0.1579200	best: 0.1579200 (0)	total: 10.5ms	remaining: 1m 45s
1000:	learn: 0.0827327	test: 0.0835241	best: 0.0835241 (1000)	total: 9.07s	remaining: 1m 21s
2000:	learn: 0.0721647	test: 0.0737235	best: 0.0737235 (2000)	total: 17.2s	remaining: 1m 8s
3000:	learn: 0.0656433	test: 0.0677148	best: 0.0677148 (3000)	total: 25s	remaining: 58.2s
4000:	learn: 0.0610588	test: 0.0634685	best: 0.0634685 (4000)	total: 32.7s	remaining: 49s
5000:	learn: 0.0573142	test: 0.0601001	best: 0.0601001 (5000)	total: 40.8s	remaining: 40.7s
6000:	learn: 0.0545693	test: 0.0576419	best: 0.0576419 (6000)	total: 48.4s	remaining: 32.3s
7000:	learn: 0.0523032	test: 0.0555463	best: 0.0555463 (7000)	total: 56.2s	remaining: 24.1s
8000:	learn: 0.0501293	test: 0.0536081	best: 0.0536081 (8000)	total: 1m 3s	remaining: 16s
9000:	learn: 0.0481672	test: 0.0518847	best: 0.0518847 (9000)	total: 1m 11s	remaining: 7.95s
9999:	learn: 0.0466431	test: 0.0505775	best: 0.0505775 (99

In [162]:
from sklearn.metrics import *
y_pred_power = model_power.predict(X_test_power)
np.sqrt(mean_squared_error(y_pred_power, y_test_power))

0.05057754339403166

In [163]:
y_test_power.max()

0.9647222222222221

In [164]:
y_pred_power.max()

0.7798484105672486

## PREDICTION kwh

In [165]:
from catboost import *
model = CatBoostRegressor(iterations=12000, loss_function='RMSE', use_best_model=True,
                          early_stopping_rounds=100, verbose=100, random_state=42,
                          )
model.fit(X_train, y_train, eval_set=(X_test, y_test))

Learning rate set to 0.023129
0:	learn: 3.7011977	test: 3.8178042	best: 3.8178042 (0)	total: 10.2ms	remaining: 2m 2s
100:	learn: 2.6448406	test: 2.7396327	best: 2.7396327 (100)	total: 783ms	remaining: 1m 32s
200:	learn: 2.3600501	test: 2.4299296	best: 2.4299296 (200)	total: 1.54s	remaining: 1m 30s
300:	learn: 2.1918257	test: 2.2558176	best: 2.2558176 (300)	total: 2.3s	remaining: 1m 29s
400:	learn: 2.0792282	test: 2.1426115	best: 2.1426115 (400)	total: 3.06s	remaining: 1m 28s
500:	learn: 1.9709730	test: 2.0373580	best: 2.0373580 (500)	total: 3.83s	remaining: 1m 27s
600:	learn: 1.8991438	test: 1.9701940	best: 1.9701940 (600)	total: 4.59s	remaining: 1m 27s
700:	learn: 1.8271443	test: 1.9044556	best: 1.9044556 (700)	total: 5.36s	remaining: 1m 26s
800:	learn: 1.7809015	test: 1.8676066	best: 1.8676066 (800)	total: 6.12s	remaining: 1m 25s
900:	learn: 1.7364015	test: 1.8309681	best: 1.8309681 (900)	total: 6.89s	remaining: 1m 24s
1000:	learn: 1.7007893	test: 1.8036446	best: 1.8036446 (1000)	tot

In [183]:
ss = pd.read_csv("SampleSubmission.csv")
ss["Date"] = pd.to_datetime(ss["ID"].str.extract(r'(\d{4}-\d{2}-\d{2})')[0])

In [184]:
# Convert to datetime
ss["Date"] = pd.to_datetime(ss["Date"])
climate_df["Date Time"] = pd.to_datetime(climate_df["Date Time"])

# Aggregate climate data to daily level
climate_daily = climate_df.groupby(climate_df["Date Time"].dt.date).agg({
    "Temperature (°C)": "mean",
    "Dewpoint Temperature (°C)": "mean",
    "U Wind Component (m/s)": "mean",
    "V Wind Component (m/s)": "mean",
    "Total Precipitation (mm)": "sum",
    "Snowfall (mm)": "sum",
    "Snow Cover (%)": "mean",
}).reset_index()

# Convert 'Date' column in climate_daily to datetime format
climate_daily.rename(columns={"Date Time": "Date"}, inplace=True)
climate_daily["Date"] = pd.to_datetime(climate_daily["Date"])  # Ensure datetime64[ns]

# Merge with complete_data
forecast = ss.merge(climate_daily, on="Date", how="left")

In [185]:
import numpy as np


# The 'year', 'day', and 'is_weekend' don't require sinusoidal encoding as they are not cyclical
# Convert 'Date' to datetime format if not already
forecast["Date"] = pd.to_datetime(forecast["Date"])

# Extract date features (keeping only useful ones)
forecast["year"] = forecast["Date"].dt.year
forecast["day"] = forecast["Date"].dt.day
forecast["is_weekend"] = (forecast["Date"].dt.dayofweek >= 5).astype(int)  # 1 if Sat/Sun, else 0

# Sinusoidal encoding of cyclical features
forecast["day_of_week_sin"] = np.sin(2 * np.pi * forecast["Date"].dt.dayofweek / 7)
forecast["day_of_week_cos"] = np.cos(2 * np.pi * forecast["Date"].dt.dayofweek / 7)

forecast["month_sin"] = np.sin(2 * np.pi * forecast["Date"].dt.month / 12)
forecast["month_cos"] = np.cos(2 * np.pi * forecast["Date"].dt.month / 12)

forecast["week_of_year_sin"] = np.sin(2 * np.pi * forecast["Date"].dt.isocalendar().week / 52)
forecast["week_of_year_cos"] = np.cos(2 * np.pi * forecast["Date"].dt.isocalendar().week / 52)

forecast["quarter_sin"] = np.sin(2 * np.pi * forecast["Date"].dt.quarter / 4)
forecast["quarter_cos"] = np.cos(2 * np.pi * forecast["Date"].dt.quarter / 4)

# Drop raw features that are now redundant


In [186]:
# Extract consumer device and data user using regex
forecast["consumer_device"] = forecast["ID"].str.extract(r'consumer_device_(\d+)_data_user_\d+').astype(int)
forecast["data_user"] = forecast["ID"].str.extract(r'consumer_device_\d+_data_user_(\d+)').astype(int)
forecast.head()

,ID,kwh,Date,Temperature (°C),Dewpoint Temperature (°C),U Wind Component (m/s),V Wind Component (m/s),Total Precipitation (mm),Snowfall (mm),Snow Cover (%),...,day_of_week_sin,day_of_week_cos,month_sin,month_cos,week_of_year_sin,week_of_year_cos,quarter_sin,quarter_cos,consumer_device,data_user
0,2024-09-24_consumer_device_12_data_user_1,0,2024-09-24,13.899341,2.104299,0.005811,-0.264604,0.000032,0.000000,0.000000,...,0.781831,0.623490,-1.0,-1.836970e-16,-1.0,-0.0,-1.0,-1.836970e-16,12,1
1,2024-09-25_consumer_device_12_data_user_1,0,2024-09-25,12.475849,5.623678,0.224280,0.183461,0.003789,0.000000,0.000000,...,0.974928,-0.222521,-1.0,-1.836970e-16,-1.0,-0.0,-1.0,-1.836970e-16,12,1
2,2024-09-26_consumer_device_12_data_user_1,0,2024-09-26,9.702699,7.375160,0.202651,0.044908,0.122680,0.000000,0.000000,...,0.433884,-0.900969,-1.0,-1.836970e-16,-1.0,-0.0,-1.0,-1.836970e-16,12,1
3,2024-09-27_consumer_device_12_data_user_1,0,2024-09-27,6.806661,5.905107,0.042285,0.168467,0.443926,0.019848,3.789225,...,-0.433884,-0.900969,-1.0,-1.836970e-16,-1.0,-0.0,-1.0,-1.836970e-16,12,1
4,2024-09-28_consumer_device_12_data_user_1,0,2024-09-28,6.399286,2.932036,-0.042425,0.149797,0.059494,0.001262,4.174235,...,-0.974928,-0.222521,-1.0,-1.836970e-16,-1.0,-0.0,-1.0,-1.836970e-16,12,1


In [187]:
X_test_final = forecast.drop(columns=['Date','ID','kwh'])
X_test_final.fillna(-1,inplace=True)

In [188]:
predictions_power = model_power.predict(X_test_final[X_train_power.columns])

In [189]:
predictions_power

array([0.50521889, 0.55318828, 0.58201941, ..., 0.51190988, 0.49695386,
       0.54540934])

In [190]:
X_test_final['power_factor']= predictions_power

In [174]:
X_test_final

,Temperature (°C),Dewpoint Temperature (°C),U Wind Component (m/s),V Wind Component (m/s),Total Precipitation (mm),Snowfall (mm),Snow Cover (%),year,day,is_weekend,...,day_of_week_cos,month_sin,month_cos,week_of_year_sin,week_of_year_cos,quarter_sin,quarter_cos,consumer_device,data_user,power_factor
0,13.899341,2.104299,0.005811,-0.264604,0.000032,0.000000,0.000000,2024,24,0,...,0.623490,-1.000000,-1.836970e-16,-1.0,-0.0,-1.000000e+00,-1.836970e-16,12,1,0.505219
1,12.475849,5.623678,0.224280,0.183461,0.003789,0.000000,0.000000,2024,25,0,...,-0.222521,-1.000000,-1.836970e-16,-1.0,-0.0,-1.000000e+00,-1.836970e-16,12,1,0.553188
2,9.702699,7.375160,0.202651,0.044908,0.122680,0.000000,0.000000,2024,26,0,...,-0.900969,-1.000000,-1.836970e-16,-1.0,-0.0,-1.000000e+00,-1.836970e-16,12,1,0.582019
3,6.806661,5.905107,0.042285,0.168467,0.443926,0.019848,3.789225,2024,27,0,...,-0.900969,-1.000000,-1.836970e-16,-1.0,-0.0,-1.000000e+00,-1.836970e-16,12,1,0.629935
4,6.399286,2.932036,-0.042425,0.149797,0.059494,0.001262,4.174235,2024,28,1,...,-0.222521,-1.000000,-1.836970e-16,-1.0,-0.0,-1.000000e+00,-1.836970e-16,12,1,0.654474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6009,3.693599,-4.460602,0.046748,-0.296137,0.003672,0.000000,2.138590,2024,20,1,...,0.623490,-0.866025,5.000000e-01,-0.935016,0.354605,-2.449294e-16,1.000000e+00,8,9,0.515363
6010,3.712101,-3.217974,-0.118125,-0.373651,0.008551,0.000000,1.033285,2024,21,0,...,1.000000,-0.866025,5.000000e-01,-0.885456,0.464723,-2.449294e-16,1.000000e+00,8,9,0.546564
6011,2.515382,-2.358298,0.047106,-0.214380,0.010362,0.000161,0.531738,2024,22,0,...,0.623490,-0.866025,5.000000e-01,-0.885456,0.464723,-2.449294e-16,1.000000e+00,8,9,0.511910
6012,0.315618,-0.711822,0.001728,-0.227600,0.056409,0.027086,6.653727,2024,23,0,...,-0.222521,-0.866025,5.000000e-01,-0.885456,0.464723,-2.449294e-16,1.000000e+00,8,9,0.496954


In [175]:
X_test_final.fillna(-1,inplace=True)

In [176]:
predictions= model.predict(X_test_final[X_train.columns])

In [177]:
forecast = pd.read_csv("SampleSubmission.csv")

In [178]:
forecast['kwh'] = predictions.clip(min=0)

In [179]:
forecast

,ID,kwh
0,2024-09-24_consumer_device_12_data_user_1,0.378923
1,2024-09-25_consumer_device_12_data_user_1,0.000000
2,2024-09-26_consumer_device_12_data_user_1,3.044177
3,2024-09-27_consumer_device_12_data_user_1,0.000000
4,2024-09-28_consumer_device_12_data_user_1,0.000000
...,...,...
6009,2024-10-20_consumer_device_8_data_user_9,0.000000
6010,2024-10-21_consumer_device_8_data_user_9,0.000000
6011,2024-10-22_consumer_device_8_data_user_9,0.283056
6012,2024-10-23_consumer_device_8_data_user_9,0.000000


In [180]:
# prompt: list the difference in the ID between forecast and ss

# Assuming 'forecast' and 'ss' DataFrames are already defined as in your provided code.

# Convert 'ID' columns to sets for efficient comparison
forecast_ids = set(forecast['ID'])
ss_ids = set(ss['ID'])

# Find IDs present in forecast but not in ss
forecast_only_ids = forecast_ids - ss_ids

# Find IDs present in ss but not in forecast
ss_only_ids = ss_ids - forecast_ids

# Print the IDs that are in forecast but not in ss
print("IDs in 'forecast' but not in 'ss':")
print(forecast_only_ids)


# Print the IDs that are in ss but not in forecast
print("\nIDs in 'ss' but not in 'forecast':")
print(ss_only_ids)

# Print the number of IDs that differ
print(f"\nNumber of IDs that differ: {len(forecast_only_ids) + len(ss_only_ids)}")


IDs in 'forecast' but not in 'ss':
set()

IDs in 'ss' but not in 'forecast':
set()

Number of IDs that differ: 0


In [181]:
forecast_best1 = pd.read_csv("forecast-cat_sin12K.csv")
forecast_b = pd.read_csv("forecast-cat-powerpred.csv")

import pandas as pd
from sklearn.metrics import mean_squared_error
import math

# Assuming 'forecast' and 'forecast_best' are DataFrames with a common 'ID' column and a 'kwh' column
# containing the forecast and actual values respectively.

# Merge the forecast and forecast_best DataFrames on the 'ID' column
merged_df = pd.merge(forecast, forecast_best1, on='ID', how='left', suffixes=('_forecast', '_actual'))

# Calculate the RMSE
rmse = math.sqrt(mean_squared_error(merged_df['kwh_actual'], merged_df['kwh_forecast']))

print(f"RMSE: {rmse}")

# Calculate the absolute difference between forecast and actual kwh values
merged_df['kwh_diff'] = abs(merged_df['kwh_actual'] - merged_df['kwh_forecast'])

# Define a threshold for "high" differences (e.g., 10% of the actual value)
threshold = 0.1  # You can adjust this based on your data and needs
merged_df['high_diff'] = merged_df['kwh_diff'] > (threshold * merged_df['kwh_actual'])

# Show rows where the difference is higher than the threshold
high_diff_rows = merged_df[merged_df['high_diff']]

# Display the rows where the difference is considered high
print("\nRows with high forecast errors:")
print(high_diff_rows[['ID', 'kwh_forecast', 'kwh_actual', 'kwh_diff']])
forecast.to_csv("forecast-cat-powerpred.csv", index = False)

RMSE: 2.972213999526346

Rows with high forecast errors:
                                             ID  kwh_forecast  kwh_actual  \
0     2024-09-24_consumer_device_12_data_user_1      0.378923    0.000000   
2     2024-09-26_consumer_device_12_data_user_1      3.044177    0.000000   
5     2024-09-29_consumer_device_12_data_user_1      3.804235    0.000000   
6     2024-09-30_consumer_device_12_data_user_1      2.264010    0.000000   
7     2024-10-01_consumer_device_12_data_user_1      4.199988    1.843480   
...                                         ...           ...         ...   
6009   2024-10-20_consumer_device_8_data_user_9      0.000000    0.484747   
6010   2024-10-21_consumer_device_8_data_user_9      0.000000    0.554929   
6011   2024-10-22_consumer_device_8_data_user_9      0.283056    0.672123   
6012   2024-10-23_consumer_device_8_data_user_9      0.000000    0.509170   
6013   2024-10-24_consumer_device_8_data_user_9      0.000000    0.631405   

      kwh_diff  
0

In [182]:
forecast

,ID,kwh
0,2024-09-24_consumer_device_12_data_user_1,0.378923
1,2024-09-25_consumer_device_12_data_user_1,0.000000
2,2024-09-26_consumer_device_12_data_user_1,3.044177
3,2024-09-27_consumer_device_12_data_user_1,0.000000
4,2024-09-28_consumer_device_12_data_user_1,0.000000
...,...,...
6009,2024-10-20_consumer_device_8_data_user_9,0.000000
6010,2024-10-21_consumer_device_8_data_user_9,0.000000
6011,2024-10-22_consumer_device_8_data_user_9,0.283056
6012,2024-10-23_consumer_device_8_data_user_9,0.000000
